In [3]:
import numpy as np 
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from lightgbm import LGBMClassifier
from preprocessing import preprocess

## Preprocess Data

In [4]:
df = pd.read_csv('data/accepted_2007_to_2018Q4.csv')
df = preprocess(df)

X, y = df.drop("loan_status", axis=1), df["loan_status"]
cols = [col for col in X.columns if X[col].dtype == "float64"]
X = X[cols]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

/var/folders/cb/pwqg37_d6zl28q9nq70dkfzw0000gn/T/ipykernel_48150/2994343687.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/accepted_2007_to_2018Q4.csv')
/Users/clundberg/Development/loan-default/preprocessing.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["earliest_cr_line"] = pd.to_datetime(df["earliest_cr_line"]).dt.year.astype(int)


## Train Classifiers

In [3]:
clf = LGBMClassifier(is_unbalance=True)
clf.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 689120, number of negative: 171878
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015070 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1994
[LightGBM] [Info] Number of data points in the train set: 860998, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.800374 -> initscore=1.388630
[LightGBM] [Info] Start training from score 1.388630


LGBMClassifier(is_unbalance=True)

## Evaluate Model

In [18]:
preds = clf.predict_proba(X_val)
precision, recall, _ = precision_recall_curve(y_val, preds[:, 1], pos_label="Fully Paid", drop_intermediate=True)

In [56]:
clf.booster_.save_model('api/lgbm_model.txt')